In [1]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [2]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

# FILE PATHS

In [15]:
FB_TOP500_ALIGNMENT_CSV_FILE = '~/Downloads/top500.csv'
OUTPUT_CSV = 'fb_align.csv'

In [4]:
import csv, pandas as pd
fb_table = pd.DataFrame.from_csv(os.path.expanduser(FB_TOP500_ALIGNMENT_CSV_FILE))


In [5]:
import mediacloud, json
mc = mediacloud.api.MediaCloud(api_key)


In [6]:
all_media = []

last_media_id = 0

while True:
    media = mc.mediaList(last_media_id = last_media_id, rows = 1000)
    print len(all_media)
    if len( media ) == 0:
        break
        
    last_media_id = media[-1]['media_id']
    all_media += media

len(all_media)
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


263217

#clean media cloud urls to enable match

In [8]:
import urlnorm
from urlparse import urlparse
all_media_clean = {}

for media in all_media:
    url = media['url']
    media_id = media['media_id']
    try:
        #matching cleaned/parsed urls from mediacloud to mediacloud media IDs.
        all_media_clean[str(urlparse(url).netloc)] = media_id
    except:
        #these urls are exceptionally broken. it is justifiable to remove them; 
        #they don't work in browsers and i believe there's maybe 3 out of 260k. 
        print url
        all_media.remove(media)
        

http://[http:/#spider
http://www.reading.org]/#spider
http://[http/
http://christian-jewishvoterguides.org]/
http://health-insurance-buyer.com]contact/
http://health-insurance-buyer.com]no/
http://[www.amazon.com/
http://wildernesscommittee.org]/
http://co2science.org]/
http://autismhelpforyou.com]http/
http://globalcommonstrust.org]/
http://]http/
http://[clickthru=broadcoalition=www.nylcv.org/
http://globalsocialjustice]/
http://38degrees.org.uk]/


In [9]:
matches = []
matched_by_url_parser = []
matched_url_parser_count = 0

for s, r in fb_table.iterrows():
    #all_media_clean is a hashtable (O(1) lookup) -- if the cleaned version of a url in the facebook dataset is there,
    #we will find it and match it to its mediacloud ID.
    
    url_path = str(urlparse(str(s)).path)
    
    if url_path in all_media_clean:
        
        match = dict( r )
        match['media_id'] = all_media_clean[ url_path]
        match['domain'] = s
        matches.append(match )
print len(matches)

    

420


In [13]:
df = pd.DataFrame( matches)
df.to_csv(os.path.expanduser(OUTPUT_CSV), 
          index=False, columns = ['media_id', 'domain', 'avg_align', 'l1','l2', 'n', 'r1', 'r2' ])


In [14]:
from IPython.display import display, HTML
HTML(df.to_html() )

,avg_align,domain,l1,l2,media_id,n,r1,r2
0,-0.9984,readersupportednews.org,0.1755,0.8215,33824,0.0028,0.0002,0.0000
1,-0.9710,act.weareultraviolet.org,0.8480,0.1520,113989,0.0000,0.0000,0.0000
2,-0.9673,www.commondreams.org,0.0945,0.8944,125311,0.0108,0.0001,0.0002
3,-0.9487,jezebel.com,0.0473,0.9498,88712,0.0026,0.0000,0.0003
4,-0.9463,www.thenation.com,0.1011,0.8956,202915,0.0030,0.0001,0.0002
5,-0.9407,www.rightwingwatch.org,0.1046,0.8874,196399,0.0060,0.0020,0.0000
6,-0.9340,www.democracynow.org,0.0401,0.9555,26521,0.0039,0.0000,0.0004
7,-0.9276,www.advocate.com,0.0350,0.9609,30129,0.0025,0.0005,0.0010
8,-0.9218,crooksandliars.com,0.0799,0.9145,26242,0.0024,0.0030,0.0002
9,-0.9097,www.newnownext.com,0.9352,0.0648,91934,0.0000,0.0000,0.0000
